**1) Install/ Import the required Python Packages/ Libraries**

In [1]:
#Import required python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")
from sklearn import preprocessing
%matplotlib inline

In [2]:
pip install category_encoders

  Obtaining dependency information for category_encoders from https://files.pythonhosted.org/packages/7f/e5/79a62e5c9c9ddbfa9ff5222240d408c1eeea4e38741a0dc8343edc7ef1ec/category_encoders-2.6.3-py2.py3-none-any.whl.metadata
  Using cached category_encoders-2.6.3-py2.py3-none-any.whl.metadata (8.0 kB)
Using cached category_encoders-2.6.3-py2.py3-none-any.whl (81 kB)
Note: you may need to restart the kernel to use updated packages.


**2) Mounting the Google Drive**

In [3]:
# Mount the Google Drive
# from google.colab import drive
# drive.mount('/content/gdrive')

**3) Read the Data file and check**

In [4]:
# Read the Diabetes Data from .csv file and check the data shape (number of Rows and Columns)
df = pd.read_csv('C:/Projects/05_Streamlit_Bigmart_Sales/Data-Files/Bigmart_sales_train1.csv')
print(df.shape)
df.head()

(2637, 12)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDY07,11.80,Low Fat,0.000000,Fruits and Vegetables,45.5402,OUT049,1999,Medium,Tier 1,Supermarket Type1,1516.0266
3,FDX32,15.10,Regular,0.100014,Fruits and Vegetables,145.4786,OUT049,1999,Medium,Tier 1,Supermarket Type1,1589.2646
4,FDP49,9.00,Regular,0.069196,Breakfast,54.3614,OUT049,1999,Medium,Tier 1,Supermarket Type1,718.3982


##**I) Check and decide the ML Learning Type and sub-type as applicable**

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            2637 non-null   object 
 1   Item_Weight                2637 non-null   float64
 2   Item_Fat_Content           2637 non-null   object 
 3   Item_Visibility            2637 non-null   float64
 4   Item_Type                  2637 non-null   object 
 5   Item_MRP                   2637 non-null   float64
 6   Outlet_Identifier          2637 non-null   object 
 7   Outlet_Establishment_Year  2637 non-null   int64  
 8   Outlet_Size                1587 non-null   object 
 9   Outlet_Location_Type       2637 non-null   object 
 10  Outlet_Type                2637 non-null   object 
 11  Item_Outlet_Sales          2637 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 247.3+ KB


In [6]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1050
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

**Observations on the given Dataset:**
* a) Number of Independet Variables: 11 (Identified)
* b) Number of Dependent Variable : 1 (Item_Outlet_Sales) (Identified)
* c) There is no Missing Value in the Dependent Variable column "Item_Outlet_Sales"


**Conclusions:**
###**a) The given dataset probably belongs to the"Supervised Learning" main-type**
###**b) Since the Dependent variable values are varying integers in nature, the given dataset is of "Regression" sub-type.**

##**II) Check and remove the duplicate records, if any**

In [7]:
df.shape

(2637, 12)

In [8]:
# Returns True for every row that is a duplicate, othwerwise False:
print(df.duplicated())

0       False
1       False
2       False
3       False
4       False
        ...  
2632    False
2633    False
2634    False
2635    False
2636    False
Length: 2637, dtype: bool


In [9]:
# Remove all duplicates:
df.drop_duplicates(inplace = True)

In [10]:
df.shape

(2637, 12)

###**Conclusion: No Duplicate Records**

##**IV) Check for Missing Values and handle them as required**

**a) Check the Missing Values, if any**

In [11]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1050
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

**b) Checking the total number of rows having the missing Values**

In [12]:
df[df.isnull().any(axis=1)]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
7,NCD06,13.000,Low Fat,0.099887,Household,45.9060,OUT017,2007,NaN,Tier 2,Supermarket Type1,838.9080
10,FDO23,17.850,Low Fat,0.000000,Breads,93.1436,OUT045,2002,NaN,Tier 2,Supermarket Type1,2174.5028
13,FDA43,10.895,Low Fat,0.065042,Fruits and Vegetables,196.3794,OUT017,2007,NaN,Tier 2,Supermarket Type1,3121.2704
14,NCP18,12.150,Low Fat,0.028760,Household,151.4708,OUT017,2007,NaN,Tier 2,Supermarket Type1,4815.0656
16,NCX54,9.195,Low Fat,0.048157,Household,106.1622,OUT045,2002,NaN,Tier 2,Supermarket Type1,2117.2440
...,...,...,...,...,...,...,...,...,...,...,...,...
2624,NCI54,15.200,Low Fat,0.000000,Household,110.4912,OUT017,2007,NaN,Tier 2,Supermarket Type1,1637.8680
2628,NCQ42,20.350,Low Fat,0.000000,Household,125.1678,OUT017,2007,NaN,Tier 2,Supermarket Type1,1907.5170
2629,FDW21,5.340,Regular,0.005998,Snack Foods,100.4358,OUT017,2007,NaN,Tier 2,Supermarket Type1,1508.0370
2630,NCH43,8.420,Low Fat,0.070712,Household,216.4192,OUT045,2002,NaN,Tier 2,Supermarket Type1,3020.0688


**c) Observations, Decisions and Actions**

**Observations:**
* a) Here, the data values of 2 columns are missing
* b) The total number rows having missing values is 3873 against the total number of rows (8523) in the dataset. 
###**So, we cannot use the option of dropping the rows having missing values.**

**Decision and Actions:**

###**Fill the missing values of the columns with that of the most_frequent values of the respective columns.**

**d) Imputation of Missing Values using the "fillna" command and checking**

In [13]:
df.fillna(df.mode().iloc[0], inplace=True)

In [14]:
df.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

##**V) Check for necessity of creating new column(s) and create the columns as required**

###**Decision: As of now, there is no necessity to create new column(s).**

##**VI) Check the unique Values of each column and observe the following and take actions as required:**
* **a) Wrong Data in the columns, if any** 
* **b) Wrong format of the data in the columns, if any**
* **c) Identify the columns which need to be categorically converted to numeric values by using Nominal method/ Ordinal Method**


###**Column-1: Item_Identifier**

In [15]:
df['Item_Identifier'].value_counts()

Item_Identifier
FDA15    5
NCU42    5
FDO32    5
FDY31    5
FDY57    5
        ..
NCA54    1
DRH15    1
FDY32    1
FDT35    1
NCD43    1
Name: count, Length: 873, dtype: int64

**Observations:**
* a) Data in this column will not be contributing to the prediction of the Depenedent variable

**Decsion:**

**We will be dropping this column before the execution of "Train_Test_Split"**

###**Column-2: Item_Weight**

In [16]:
df['Item_Weight'].value_counts()

Item_Weight
17.600    34
18.250    30
9.600     29
13.150    29
16.000    28
          ..
6.570      1
6.635      1
8.000      1
17.200     1
6.440      1
Name: count, Length: 311, dtype: int64

**Observations:**
* a) Data in this column is of "float" datatype. 

**Decsion:**

**So, we will keep this column as it is.**

###**Column-3: Item_Fat_Content**

In [17]:
df['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    1613
Regular     844
LF          102
reg          41
low fat      37
Name: count, dtype: int64

**Observations:**
* a) Data in this column is of "Object" or "String" datatype. The labels are of ordinal type. So we need to use label encoder. 
* b) Also, the data levels need to be modified like "low fat" and "LF" to "Low Fat", "reg" to "Regular".

**Decision:**
**We will be converting the data (after correcting the wrong labels) in this column into Numerical values using Ordnial Type method "preprocessing.LabelEncoder()**


In [18]:
df.replace("LF", "Low Fat", inplace=True)
df.replace("low fat", "Low Fat", inplace=True)
df.replace("reg", "Regular", inplace=True)
df['Item_Fat_Content'].value_counts()

Item_Fat_Content
Low Fat    1752
Regular     885
Name: count, dtype: int64

In [19]:
le = preprocessing.LabelEncoder()
df['Item_Fat_Content'] = le.fit_transform(df.Item_Fat_Content.values)
df['Item_Fat_Content'].value_counts()

Item_Fat_Content
0    1752
1     885
Name: count, dtype: int64

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            2637 non-null   object 
 1   Item_Weight                2637 non-null   float64
 2   Item_Fat_Content           2637 non-null   int32  
 3   Item_Visibility            2637 non-null   float64
 4   Item_Type                  2637 non-null   object 
 5   Item_MRP                   2637 non-null   float64
 6   Outlet_Identifier          2637 non-null   object 
 7   Outlet_Establishment_Year  2637 non-null   int64  
 8   Outlet_Size                2637 non-null   object 
 9   Outlet_Location_Type       2637 non-null   object 
 10  Outlet_Type                2637 non-null   object 
 11  Item_Outlet_Sales          2637 non-null   float64
dtypes: float64(4), int32(1), int64(1), object(6)
memory usage: 237.0+ KB


###**Column-4: Item_Visibility**

In [21]:
df['Item_Visibility'].value_counts()

Item_Visibility
0.000000    168
0.076975      3
0.059160      2
0.052150      2
0.080131      2
           ... 
0.020580      1
0.025912      1
0.115347      1
0.033667      1
0.145221      1
Name: count, Length: 2435, dtype: int64

**Observations:**
* a) Data in this column is of "float" datatype. Also, there seems to be no wrong data or wrong data format.

**Decsion:**

**Let us leave the column as it is.**

###**Column-5: Item_Type**

In [22]:
df['Item_Type'].value_counts()

Item_Type
Fruits and Vegetables    672
Snack Foods              672
Household                486
Dairy                    365
Soft Drinks              249
Breads                   136
Breakfast                 57
Name: count, dtype: int64

**Observations:**
* a) Data in this column is of "Object" or "String" datatype. Also, the data levels are "Nominal" Type.

**Decsion:**

**We will be converting the data in this column into Numerical values using Nominal Type method "Binary Encoding" as we want to have minimum number of columns.**

In [23]:
#Create object for binary encoding
import category_encoders as ce
encoder1= ce.BinaryEncoder(cols=['Item_Type'],return_df=True)

In [24]:
#Fit and Transform Data 
itemtype_encoded=encoder1.fit_transform(df['Item_Type']) 
itemtype_encoded

,Item_Type_0,Item_Type_1,Item_Type_2
0,0,0,1
1,0,1,0
2,0,1,1
3,0,1,1
4,1,0,0
...,...,...,...
2632,1,1,0
2633,0,1,0
2634,0,1,1
2635,1,1,0


###**Column-6: Outlet_Identifier**

In [25]:
df['Outlet_Identifier'].value_counts()

Outlet_Identifier
OUT045    538
OUT049    534
OUT035    533
OUT018    520
OUT017    512
Name: count, dtype: int64

**Observations:**
* a) Data in this column is of "Object" or "String" datatype. Also, the data levels are "Nominal" Type.

**Decsion:**

**We will be converting the data in this column into Numerical values using Nominal Type method "Binary Encoding" as we want to have minimum number of columns.**

In [26]:
#Create object for binary encoding
import category_encoders as ce
encoder2= ce.BinaryEncoder(cols=['Outlet_Identifier'],return_df=True)

In [27]:
#Fit and Transform Data 
outletidentifier_encoded=encoder2.fit_transform(df['Outlet_Identifier']) 
outletidentifier_encoded

,Outlet_Identifier_0,Outlet_Identifier_1,Outlet_Identifier_2
0,0,0,1
1,0,1,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
2632,0,1,0
2633,0,1,0
2634,1,0,1
2635,0,1,1


###**Column-7: Outlet_Establishment_Year**

In [28]:
df['Outlet_Establishment_Year'].value_counts()

Outlet_Establishment_Year
2002    538
1999    534
2004    533
2009    520
2007    512
Name: count, dtype: int64

**Observations:**
* a) Data in this column is of "int" datatype. Also, there seems to be no wrong data or wrong data format.

**Decsion:**

**Let us leave the column as it is.**

###**Column-8: Outlet_Size**

In [29]:
df['Outlet_Size'].value_counts()

Outlet_Size
Medium    2104
Small      533
Name: count, dtype: int64

**Observations:**
* a) Data in this column is of "Object" or "String" datatype. Also, the data levels are "Ordinal" Type.

**Decsion:**

**We will be converting the data in this column into Numerical values using Ordnial Type method "preprocessing.LabelEncoder()**

In [30]:
le = preprocessing.LabelEncoder()
df['Outlet_Size'] = le.fit_transform(df.Outlet_Size.values)
df['Outlet_Size'].value_counts()

Outlet_Size
0    2104
1     533
Name: count, dtype: int64

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            2637 non-null   object 
 1   Item_Weight                2637 non-null   float64
 2   Item_Fat_Content           2637 non-null   int32  
 3   Item_Visibility            2637 non-null   float64
 4   Item_Type                  2637 non-null   object 
 5   Item_MRP                   2637 non-null   float64
 6   Outlet_Identifier          2637 non-null   object 
 7   Outlet_Establishment_Year  2637 non-null   int64  
 8   Outlet_Size                2637 non-null   int32  
 9   Outlet_Location_Type       2637 non-null   object 
 10  Outlet_Type                2637 non-null   object 
 11  Item_Outlet_Sales          2637 non-null   float64
dtypes: float64(4), int32(2), int64(1), object(5)
memory usage: 226.7+ KB


###**Column-8: Outlet_Location_Type**

In [32]:
df['Outlet_Location_Type'].value_counts()

Outlet_Location_Type
Tier 2    1583
Tier 1     534
Tier 3     520
Name: count, dtype: int64

**Observations:**
* a) Data in this column is of "Object" or "String" datatype. Also, the data levels are "Ordinal" Type.

**Decsion:**

**We will be converting the data in this column into Numerical values using Ordnial Type method "preprocessing.LabelEncoder()**

In [33]:
le = preprocessing.LabelEncoder()
df['Outlet_Location_Type'] = le.fit_transform(df.Outlet_Location_Type.values)
df['Outlet_Location_Type'].value_counts()

Outlet_Location_Type
1    1583
0     534
2     520
Name: count, dtype: int64

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            2637 non-null   object 
 1   Item_Weight                2637 non-null   float64
 2   Item_Fat_Content           2637 non-null   int32  
 3   Item_Visibility            2637 non-null   float64
 4   Item_Type                  2637 non-null   object 
 5   Item_MRP                   2637 non-null   float64
 6   Outlet_Identifier          2637 non-null   object 
 7   Outlet_Establishment_Year  2637 non-null   int64  
 8   Outlet_Size                2637 non-null   int32  
 9   Outlet_Location_Type       2637 non-null   int32  
 10  Outlet_Type                2637 non-null   object 
 11  Item_Outlet_Sales          2637 non-null   float64
dtypes: float64(4), int32(3), int64(1), object(4)
memory usage: 216.4+ KB


###**Column-10: Outlet_Type**

In [35]:
df['Outlet_Type'].value_counts()

Outlet_Type
Supermarket Type1    2117
Supermarket Type2     520
Name: count, dtype: int64

**Observations:**
* a) Data in this column is of "Object" or "String" datatype. Also, the data levels are "Ordinal" Type.

**Decsion:**

**We will be converting the data in this column into Numerical values using Ordnial Type method "preprocessing.LabelEncoder()**

In [36]:
le = preprocessing.LabelEncoder()
df['Outlet_Type'] = le.fit_transform(df.Outlet_Type.values)
df['Outlet_Type'].value_counts()

Outlet_Type
0    2117
1     520
Name: count, dtype: int64

###**Column-11: Item_Outlet_Sales**

In [37]:
df['Item_Outlet_Sales'].value_counts()

Item_Outlet_Sales
703.0848     10
1230.3984     9
1406.1696     7
1342.2528     7
1701.7848     6
             ..
1705.1138     1
3709.8376     1
3761.7700     1
2137.2180     1
7182.6504     1
Name: count, Length: 1710, dtype: int64

**Observations:**
* a) Data in this column is of "float" datatype. Also, there seems to be no wrong data or wrong data format.

**Decsion:**

**Let us leave the column as it is.**

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            2637 non-null   object 
 1   Item_Weight                2637 non-null   float64
 2   Item_Fat_Content           2637 non-null   int32  
 3   Item_Visibility            2637 non-null   float64
 4   Item_Type                  2637 non-null   object 
 5   Item_MRP                   2637 non-null   float64
 6   Outlet_Identifier          2637 non-null   object 
 7   Outlet_Establishment_Year  2637 non-null   int64  
 8   Outlet_Size                2637 non-null   int32  
 9   Outlet_Location_Type       2637 non-null   int32  
 10  Outlet_Type                2637 non-null   int32  
 11  Item_Outlet_Sales          2637 non-null   float64
dtypes: float64(4), int32(4), int64(1), object(3)
memory usage: 206.1+ KB


In [39]:
df.drop(['Item_Identifier', 'Item_Type','Outlet_Identifier'], axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                2637 non-null   float64
 1   Item_Fat_Content           2637 non-null   int32  
 2   Item_Visibility            2637 non-null   float64
 3   Item_MRP                   2637 non-null   float64
 4   Outlet_Establishment_Year  2637 non-null   int64  
 5   Outlet_Size                2637 non-null   int32  
 6   Outlet_Location_Type       2637 non-null   int32  
 7   Outlet_Type                2637 non-null   int32  
 8   Item_Outlet_Sales          2637 non-null   float64
dtypes: float64(4), int32(4), int64(1)
memory usage: 144.3 KB


In [40]:
df = pd.concat([df,itemtype_encoded, outletidentifier_encoded], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                2637 non-null   float64
 1   Item_Fat_Content           2637 non-null   int32  
 2   Item_Visibility            2637 non-null   float64
 3   Item_MRP                   2637 non-null   float64
 4   Outlet_Establishment_Year  2637 non-null   int64  
 5   Outlet_Size                2637 non-null   int32  
 6   Outlet_Location_Type       2637 non-null   int32  
 7   Outlet_Type                2637 non-null   int32  
 8   Item_Outlet_Sales          2637 non-null   float64
 9   Item_Type_0                2637 non-null   int64  
 10  Item_Type_1                2637 non-null   int64  
 11  Item_Type_2                2637 non-null   int64  
 12  Outlet_Identifier_0        2637 non-null   int64  
 13  Outlet_Identifier_1        2637 non-null   int64

In [41]:
df.describe()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Item_Type_0,Item_Type_1,Item_Type_2,Outlet_Identifier_0,Outlet_Identifier_1,Outlet_Identifier_2
count,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000
mean,12.907120,0.335609,0.061823,144.612365,2004.147895,0.202124,0.994691,0.197194,2320.103186,0.512325,0.655669,0.629124,0.398180,0.399317,0.608646
std,4.655182,0.472292,0.045105,62.460494,3.541019,0.401660,0.632313,0.397955,1534.340669,0.499943,0.475240,0.483131,0.489616,0.489851,0.488146
min,4.590000,0.000000,0.000000,31.890000,1999.000000,0.000000,0.000000,0.000000,69.243200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.770000,0.000000,0.026226,97.904200,2002.000000,0.000000,1.000000,0.000000,1174.471200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,12.850000,0.000000,0.051928,146.010200,2004.000000,0.000000,1.000000,0.000000,1965.441600,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
75%,17.000000,1.000000,0.091190,188.424000,2007.000000,0.000000,1.000000,0.000000,3135.252200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,21.350000,1.000000,0.186826,266.188400,2009.000000,1.000000,2.000000,1.000000,9664.752800,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [42]:
X = df.drop(['Item_Outlet_Sales'], axis = 1)
Y = df['Item_Outlet_Sales']
X.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Type_0,Item_Type_1,Item_Type_2,Outlet_Identifier_0,Outlet_Identifier_1,Outlet_Identifier_2
0,9.30,0,0.016047,249.8092,1999,0,0,0,0,0,1,0,0,1
1,5.92,1,0.019278,48.2692,2009,0,2,1,0,1,0,0,1,0
2,11.80,0,0.000000,45.5402,1999,0,0,0,0,1,1,0,0,1
3,15.10,1,0.100014,145.4786,1999,0,0,0,0,1,1,0,0,1
4,9.00,1,0.069196,54.3614,1999,0,0,0,1,0,0,0,0,1


In [43]:
Y.head()

0    3735.1380
1     443.4228
2    1516.0266
3    1589.2646
4     718.3982
Name: Item_Outlet_Sales, dtype: float64

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 66)

In [45]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2109, 14)
(2109,)
(528, 14)
(528,)


In [46]:
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
model=regressor.fit(X_train, y_train)

In [47]:
# Calculate the accuracy of learning by the Algorithm
regressor.score(X_train,y_train) 

0.4420620121819755

In [48]:
train_pred=regressor.predict(X_train)

In [49]:
# Predicting the Test set results
y_pred = regressor.predict(X_test)

In [50]:
# Calcualte the accuracy of prediction (y_test Vs y_pred) # This measure is part of Model Evaluation
r2_score(y_test, y_pred)

0.4675373213272769

In [51]:
regressor.score(X_test,y_test) 

0.4675373213272769

In [52]:
# Calculate the Model Error (RMSE) # This measure is also part of Model Evaluation
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

1145.880798277016

In [53]:
print("Training Accuracy :", regressor.score(X_train, y_train))
print("Testing Accuracy :", r2_score(y_test,y_pred))

Training Accuracy : 0.4420620121819755
Testing Accuracy : 0.4675373213272769


In [54]:
print("Training Error :", np.sqrt(mean_squared_error(y_train, train_pred)))
print("Testing Error :", np.sqrt(mean_squared_error(y_test, y_pred)))

Training Error : 1138.0382194272593
Testing Error : 1145.880798277016


##**VIII) Implement the Scaling as required**

###**Use Normalization**

In [55]:
X_train.columns

Index(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Type_0', 'Item_Type_1', 'Item_Type_2',
       'Outlet_Identifier_0', 'Outlet_Identifier_1', 'Outlet_Identifier_2'],
      dtype='object')

In [56]:
columnNames=(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_MRP',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Type_0', 'Item_Type_1',
       'Item_Type_2', 'Outlet_Identifier_0', 'Outlet_Identifier_1',
       'Outlet_Identifier_2'])

In [57]:
min_max_scaler_object = preprocessing.MinMaxScaler()
X_train1 = min_max_scaler_object.fit_transform(X_train)
X_train1 = pd.DataFrame(X_train1 , columns = columnNames)
X_train1.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Type_0,Item_Type_1,Item_Type_2,Outlet_Identifier_0,Outlet_Identifier_1,Outlet_Identifier_2
0,0.570406,0.0,0.193480,0.031685,0.8,0.0,0.5,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.258950,0.0,0.000000,0.091214,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.412291,0.0,0.000000,0.373978,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
3,0.785203,0.0,0.078027,0.549881,0.5,1.0,0.5,0.0,0.0,1.0,1.0,0.0,1.0,1.0
4,0.656921,1.0,0.563499,0.603446,0.5,1.0,0.5,0.0,1.0,1.0,0.0,0.0,1.0,1.0


In [58]:
min_max_scaler_object = preprocessing.MinMaxScaler()
X_test1 = min_max_scaler_object.fit_transform(X_test)
X_test1 = pd.DataFrame(X_test1 , columns = columnNames)
X_test1.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Type_0,Item_Type_1,Item_Type_2,Outlet_Identifier_0,Outlet_Identifier_1,Outlet_Identifier_2
0,0.840936,0.0,0.068005,0.378233,0.3,0.0,0.5,0.0,1.0,0.0,1.0,1.0,0.0,1.0
1,0.624850,0.0,0.249121,0.380827,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0
2,0.354742,1.0,0.058993,0.566052,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,0.294718,0.0,0.168127,0.334054,0.3,0.0,0.5,0.0,0.0,1.0,1.0,1.0,0.0,1.0
4,0.750900,0.0,0.000000,0.351188,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


In [59]:
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
regressor1 = LinearRegression()
model1=regressor1.fit(X_train1, y_train)

In [60]:
train_pred1=regressor1.predict(X_train1)

In [61]:
# Predicting the Test set results
y_pred1 = regressor1.predict(X_test1)

In [62]:
print("Training Accuracy :", regressor1.score(X_train1,y_train))
print("Testing Accuracy :", r2_score(y_test,y_pred1))

Training Accuracy : 0.4416652266562988
Testing Accuracy : 0.46929612766248985


In [63]:
print("Training Error :", np.sqrt(mean_squared_error(y_train, train_pred1)))
print("Testing Error :", np.sqrt(mean_squared_error(y_test, y_pred1)))

Training Error : 1138.4428135297555
Testing Error : 1143.9867223893614


###**Use Standardization**

In [64]:
from sklearn.preprocessing import StandardScaler
std_scaler_object = preprocessing.StandardScaler()
X_train2 = std_scaler_object.fit_transform(X_train)
X_train2 = pd.DataFrame(X_train2 , columns = columnNames)
X_train2.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Type_0,Item_Type_1,Item_Type_2,Outlet_Identifier_0,Outlet_Identifier_1,Outlet_Identifier_2
0,0.281100,-0.726054,-0.562087,-1.672462,0.801886,-0.503110,0.007513,-0.494957,-1.015772,0.716937,-1.284328,1.225471,-0.814401,-1.240123
1,-0.840700,-0.726054,-1.363571,-1.447555,-1.459424,-0.503110,-1.577055,-0.494957,0.984473,-1.394823,0.778618,-0.816013,-0.814401,0.806371
2,-0.288396,-0.726054,-1.363571,-0.379249,1.367213,-0.503110,1.592081,2.020378,0.984473,0.716937,-1.284328,-0.816013,1.227896,-1.240123
3,1.054754,-0.726054,-1.040348,0.285329,-0.046105,1.987638,0.007513,-0.494957,-1.015772,0.716937,0.778618,-0.816013,1.227896,0.806371
4,0.592711,1.377308,0.970705,0.487702,-0.046105,1.987638,0.007513,-0.494957,0.984473,0.716937,-1.284328,-0.816013,1.227896,0.806371


In [65]:
from sklearn.preprocessing import StandardScaler
std_scaler_object = preprocessing.StandardScaler()
X_test2 = std_scaler_object.fit_transform(X_test)
X_test2 = pd.DataFrame(X_test2 , columns = columnNames)
X_test2.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Type_0,Item_Type_1,Item_Type_2,Outlet_Identifier_0,Outlet_Identifier_1,Outlet_Identifier_2
0,1.168457,-0.650523,-1.119541,-0.470271,-0.587927,-0.50414,0.011901,-0.498224,0.941124,-1.322876,0.725267,1.245281,-0.819076,0.784015
1,0.394172,-0.650523,-0.370109,-0.460861,-1.433006,-0.50414,-1.558972,-0.498224,0.941124,0.755929,0.725267,-0.803032,-0.819076,0.784015
2,-0.573686,1.537224,-1.156833,0.211059,-1.433006,-0.50414,-1.558972,-0.498224,0.941124,0.755929,-1.378803,-0.803032,-0.819076,0.784015
3,-0.788765,-0.650523,-0.705251,-0.630533,-0.587927,-0.50414,0.011901,-0.498224,-1.062559,0.755929,0.725267,1.245281,-0.819076,0.784015
4,0.845838,-0.650523,-1.400937,-0.568377,1.383923,-0.50414,1.582773,2.007130,0.941124,-1.322876,0.725267,-0.803032,1.220887,-1.275486


In [66]:
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
regressor2 = LinearRegression()
model2=regressor2.fit(X_train2, y_train)

In [67]:
train_pred2=regressor2.predict(X_train2)

In [68]:
# Predicting the Test set results
y_pred2 = regressor2.predict(X_test2)

In [69]:
print("Training Accuracy :", regressor2.score(X_train2,y_train))
print("Testing Accuracy :", r2_score(y_test,y_pred2))

Training Accuracy : 0.4418775100621666
Testing Accuracy : -1.1411876564953983e+22


In [70]:
print("Training Error :", np.sqrt(mean_squared_error(y_train, train_pred2)))
print("Testing Error :", np.sqrt(mean_squared_error(y_test, y_pred2)))

Training Error : 1138.2263703623125
Testing Error : 167754194642076.84


###**By scaling the data we have not got any improvement in the Accuracies and Errors. However, we will use the scaled data using "Normalization" method as it will be easy to process the scaled data in the future processes.**

In [71]:
X1 = df.drop(['Item_Outlet_Sales'], axis = 1)
Y1 = df['Item_Outlet_Sales']
X1.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Type_0,Item_Type_1,Item_Type_2,Outlet_Identifier_0,Outlet_Identifier_1,Outlet_Identifier_2
0,9.30,0,0.016047,249.8092,1999,0,0,0,0,0,1,0,0,1
1,5.92,1,0.019278,48.2692,2009,0,2,1,0,1,0,0,1,0
2,11.80,0,0.000000,45.5402,1999,0,0,0,0,1,1,0,0,1
3,15.10,1,0.100014,145.4786,1999,0,0,0,0,1,1,0,0,1
4,9.00,1,0.069196,54.3614,1999,0,0,0,1,0,0,0,0,1


In [72]:
min_max_scaler_object = preprocessing.MinMaxScaler()
X2 = min_max_scaler_object.fit_transform(X1)
X2 = pd.DataFrame(X2 , columns = columnNames)
print(X2.shape)
X2.head()

(2637, 14)


,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Type_0,Item_Type_1,Item_Type_2,Outlet_Identifier_0,Outlet_Identifier_1,Outlet_Identifier_2
0,0.281026,0.0,0.085894,0.930093,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.079356,1.0,0.103188,0.069907,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.430191,0.0,0.000000,0.058260,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,0.627088,1.0,0.535330,0.484803,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,0.263126,1.0,0.370379,0.095909,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [73]:
df1 = pd.DataFrame(data=X2)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                2637 non-null   float64
 1   Item_Fat_Content           2637 non-null   float64
 2   Item_Visibility            2637 non-null   float64
 3   Item_MRP                   2637 non-null   float64
 4   Outlet_Establishment_Year  2637 non-null   float64
 5   Outlet_Size                2637 non-null   float64
 6   Outlet_Location_Type       2637 non-null   float64
 7   Outlet_Type                2637 non-null   float64
 8   Item_Type_0                2637 non-null   float64
 9   Item_Type_1                2637 non-null   float64
 10  Item_Type_2                2637 non-null   float64
 11  Outlet_Identifier_0        2637 non-null   float64
 12  Outlet_Identifier_1        2637 non-null   float64
 13  Outlet_Identifier_2        2637 non-null   float

In [74]:
df1.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Type_0,Item_Type_1,Item_Type_2,Outlet_Identifier_0,Outlet_Identifier_1,Outlet_Identifier_2
0,0.281026,0.0,0.085894,0.930093,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.079356,1.0,0.103188,0.069907,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.430191,0.0,0.000000,0.058260,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
3,0.627088,1.0,0.535330,0.484803,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,0.263126,1.0,0.370379,0.095909,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [75]:
df1 = pd.concat([df1,Y1], axis=1)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                2637 non-null   float64
 1   Item_Fat_Content           2637 non-null   float64
 2   Item_Visibility            2637 non-null   float64
 3   Item_MRP                   2637 non-null   float64
 4   Outlet_Establishment_Year  2637 non-null   float64
 5   Outlet_Size                2637 non-null   float64
 6   Outlet_Location_Type       2637 non-null   float64
 7   Outlet_Type                2637 non-null   float64
 8   Item_Type_0                2637 non-null   float64
 9   Item_Type_1                2637 non-null   float64
 10  Item_Type_2                2637 non-null   float64
 11  Outlet_Identifier_0        2637 non-null   float64
 12  Outlet_Identifier_1        2637 non-null   float64
 13  Outlet_Identifier_2        2637 non-null   float

In [76]:
df1.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Type_0,Item_Type_1,Item_Type_2,Outlet_Identifier_0,Outlet_Identifier_1,Outlet_Identifier_2,Item_Outlet_Sales
0,0.281026,0.0,0.085894,0.930093,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3735.1380
1,0.079356,1.0,0.103188,0.069907,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,443.4228
2,0.430191,0.0,0.000000,0.058260,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1516.0266
3,0.627088,1.0,0.535330,0.484803,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1589.2646
4,0.263126,1.0,0.370379,0.095909,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,718.3982


In [77]:
#from google.colab import files
df1.to_csv("C:/Users/lkant/Documents/05_Streamlit_Bigmart_Sales/Data-Files/Bigmart_sales_train_preprocessed1.csv", index = False)

OSError: Cannot save file into a non-existent directory: 'C:\Users\lkant\Documents\05_Streamlit_Bigmart_Sales\Data-Files'